In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
auto = pd.read_json("scout_car.json", lines=True)
pd.set_option("display.max.columns", None)
auto.head(3)

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,kW,hp,Type,Previous Owners,Next Inspection,Inspection new,Warranty,Full Service,Non-smoking Vehicle,null,Make,Model,Offer Number,First Registration,Body Color,Paint Type,Body Color Original,Upholstery,Body,Nr. of Doors,Nr. of Seats,Model Code,Gearing Type,Displacement,Cylinders,Weight,Drive chain,Fuel,Consumption,CO2 Emission,Emission Class,\nComfort & Convenience\n,\nEntertainment & Media\n,\nExtras\n,\nSafety & Security\n,description,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,NaN,66 kW,"[, Used, , Diesel (Particulate Filter)]",\n2\n,"[\n06/2021\n, \n99 g CO2/km (comb)\n]","[\nYes\n, \nEuro 6\n]","[\n, \n, \n4 (Green)\n]","[\n, \n]","[\n, \n]",[],\nAudi\n,"[\n, A1, \n]",[\nLR-062483\n],"[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nMythosschwarz\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n5\n],[\n5\n],[\n0588/BDF\n],"[\n, Automatic, \n]","[\n1,422 cc\n]",[\n3\n],"[\n1,220 kg\n]",[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Armrest, Automatic climate ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Catalytic Converter, Voice Cont...","[ABS, Central door lock, Daytime running light...","[\n, Sicherheit:, , Deaktivierung für Beifahr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,None,NaN,141 kW,"[, Used, , Gasoline]",NaN,NaN,NaN,NaN,NaN,NaN,[],\nAudi\n,"[\n, A1, \n]",NaN,"[\n, 2017, \n]","[\n, Red, \n]",NaN,NaN,"[\nCloth, Grey\n]","[\n, Sedans, \n]",[\n3\n],[\n4\n],[\n0588/BCY\n],"[\n, Automatic, \n]","[\n1,798 cc\n]",[\n4\n],"[\n1,255 kg\n]",[\nfront\n],"[\n, Gasoline, \n]","[[5.6 l/100 km (comb)], [7.1 l/100 km (city)],...",[\n129 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Automatic climate control, ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Sport seats, Sport suspension, ...","[ABS, Central door lock, Central door lock wit...",[\nLangstreckenfahrzeug daher die hohe Kilomet...,[\n4 (Green)\n],[\n7\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,"83,450 km",02/2016,1 previous owner,NaN,85 kW,"[, Used, , Diesel (Particulate Filter)]",\n1\n,NaN,NaN,"[\n, \n, \n99 g CO2/km (comb)\n]",NaN,NaN,[],\nAudi\n,"[\n, A1, \n]",[\nAM-95365\n],"[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nmythosschwarz metallic\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n4\n],[\n4\n],NaN,"[\n, Automatic, \n]","[\n1,598 cc\n]",NaN,NaN,[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.4 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Cruise control, Electrical ...","[MP3, On-board computer]","[Alloy wheels, Voice Control]","[ABS, Central door lock, Daytime running light...","[\n, Fahrzeug-Nummer: AM-95365, , Ehem. UPE 2...",[\n4 (Green)\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 54 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   url                            15919 non-null  object 
 1   make_model                     15919 non-null  object 
 2   short_description              15873 non-null  object 
 3   body_type                      15859 non-null  object 
 4   price                          15919 non-null  int64  
 5   vat                            11406 non-null  object 
 6   km                             15919 non-null  object 
 7   registration                   15919 non-null  object 
 8   prev_owner                     9091 non-null   object 
 9   kW                             0 non-null      float64
 10  hp                             15919 non-null  object 
 11  Type                           15917 non-null  object 
 12  Previous Owners                9279 non-null  

In [4]:
auto.shape

(15919, 54)

#### We have to notice missing values. If a column has a great number of missing values, our process on that column can change our decision. I will drop columns which have over 90% percent missing values.

In [5]:
auto.isnull().sum() / len(auto) * 100

url                                0.000000
make_model                         0.000000
short_description                  0.288963
body_type                          0.376908
price                              0.000000
vat                               28.349771
km                                 0.000000
registration                       0.000000
prev_owner                        42.892141
kW                               100.000000
hp                                 0.000000
Type                               0.012564
Previous Owners                   41.711163
Next Inspection                   77.793831
Inspection new                    75.299956
Warranty                          34.047365
Full Service                      48.395000
Non-smoking Vehicle               54.915510
null                               0.000000
Make                               0.000000
Model                              0.000000
Offer Number                      19.944720
First Registration              

In [6]:
missing = (auto.isnull().sum()/ len(auto) * 100).index

In [7]:
for i in missing:
    if (auto[i].isnull().sum()/ len(auto[i]) * 100) > 90.0:
        auto.drop(i,axis=1, inplace=True)

In [8]:
auto.shape

(15919, 47)

#### "make_model"

In [9]:
auto["make_model"].value_counts()

Audi A3           3097
Audi A1           2614
Opel Insignia     2598
Opel Astra        2526
Opel Corsa        2219
Renault Clio      1839
Renault Espace     991
Renault Duster      34
Audi A2              1
Name: make_model, dtype: int64

There is only one "Audi A2". We can focus on other make_model values.

In [10]:
auto[auto["make_model"]=="Audi A2"]

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,hp,Type,Previous Owners,Next Inspection,Inspection new,Warranty,Full Service,Non-smoking Vehicle,null,Make,Model,Offer Number,First Registration,Body Color,Paint Type,Body Color Original,Upholstery,Body,Nr. of Doors,Nr. of Seats,Model Code,Gearing Type,Displacement,Cylinders,Weight,Drive chain,Fuel,Consumption,CO2 Emission,Emission Class,\nComfort & Convenience\n,\nEntertainment & Media\n,\nExtras\n,\nSafety & Security\n,description,Emission Label,Gears,Country version
2614,https://www.autoscout24.com//offers/audi-a2-30...,Audi A2,"30 TDI Navi, Lane assist, Sitzheizung, Einpark...",Off-Road,28200,VAT deductible,"26,166 km",10/2018,1 previous owner,85 kW,"[, Employee's car, , Diesel (Particulate Filter)]",\n1\n,"[\n10/2019\n, \nEuro 6\n]",NaN,"[\n, \n, \n4 (Green)\n]","[\n, \n]","[\n, \n]",[],\nAudi\n,"[\n, A2, \n]",[\n27609\n],"[\n, 2018, \n]","[\n, Grey, \n]",[\nMetallic\n],[\nG3 Nanograu Metallic\n],"[\nCloth, Black\n]","[\n, Off-Road, \n]",[\n5\n],[\n5\n],[\n0588/BJV\n],"[\n, Manual, \n]","[\n1,598 cc\n]",NaN,NaN,[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[4.9 l/100 km (comb)], [5.6 l/100 km (city)],...",NaN,[\nEuro 6\n],"[Air conditioning, Armrest, Automatic climate ...","[Bluetooth, CD player, Hands-free equipment, M...",[Alloy wheels],"[ABS, Adaptive Cruise Control, Central door lo...","[\n, Technik & Sicherheit:, 7-Gang-Automatikge...",[\n4 (Green)\n],NaN,[\nGermany\n]


In [11]:
auto.drop(2614,inplace=True)

#### "body_type"

In [12]:
auto["body_type"].value_counts(dropna=False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           55
Coupe              25
Convertible         8
Name: body_type, dtype: int64

#### "price"

In [13]:
auto["price"].sort_values()

8594        13
8828       120
6066       255
8829       331
8827      4950
         ...  
15828    64332
3587     64900
3649     67600
15826    68320
3648     74600
Name: price, Length: 15918, dtype: int64

In "price" column, some values are not logical. Because of that they will be dropped.

In [14]:
auto.drop([8594,8828,6066,8829], inplace=True)

#### "vat"

In [15]:
auto["vat"].value_counts(dropna=False)

VAT deductible      10979
NaN                  4509
Price negotiable      426
Name: vat, dtype: int64

#### "km" column has "-" values and "," will cause a problem in the future when I will try to change type.

In [16]:
auto["km"] = auto["km"].str.replace(",", "").str.strip(" km")
auto["km"] = auto["km"].replace("-",np.nan)
auto["km"].astype(float)

0        56013.0
1        80000.0
2        83450.0
3        73000.0
4        16200.0
          ...   
15914        NaN
15915     9900.0
15916       15.0
15917       10.0
15918        NaN
Name: km, Length: 15914, dtype: float64

#### "registration"

In [17]:
auto["registration"] = auto["registration"].apply(lambda x:np.nan if re.findall("\d",x) == list() else x)
auto["registration"] = pd.to_datetime(auto["registration"], format="%m/%Y")

#### "prev_owner" and "Previous Owners" columns indicate the same value. But there are slight differences between them.Previous Owner has less NaN values.

In [18]:
auto["Previous Owners"] = [i[0] if type(i) == list else i for i in auto["Previous Owners"]]
auto["Previous Owners"] = auto["Previous Owners"].str.strip("\n")

#### "hp" column doesn't enable us to change type.

In [19]:
auto["hp(kW)"] = auto["hp"].str.extract("(\d+)")
auto["hp(kW)"] = pd.to_numeric(auto["hp(kW)"])
auto.drop("hp", axis=1, inplace=True)

#### "Type" column has list and meaningles values.

In [20]:
auto["condition_type"] = auto["Type"].str[1]
auto.drop("Type", inplace=True, axis=1)

#### "Next Inspection" column has list and meaningles values.

In [21]:
auto["Next Inspection"] = ["".join(i) if type(i) == list else i for i in auto["Next Inspection"]]
auto["Next Inspection"] = auto["Next Inspection"].str.extract("(\d+/\d+)")
auto["Next Inspection"] = pd.to_datetime(auto["Next Inspection"], format="%m/%Y")

#### "Inspection_new" column has list and meaningles values. Besides it can be converted into 0 and 1.

In [22]:
auto["Inspection new"] = [i[0] if type(i) == list else i for i in auto["Inspection new"]]
auto["Inspection new"] = auto["Inspection new"].str.strip("\n")
auto["Inspection new"] = [1 if i=="Yes" else 0 for i in auto["Inspection new"]]

#### "Warranty" column has list and meaningles values. Besides it can be converted into 0 and 1.

In [23]:
auto["Warranty"] = [i[0] if type(i) == list else i for i in auto["Warranty"]]
auto["Warranty(month)"] = auto["Warranty"].str.strip(" months\n")
auto["Warranty(month)"].replace("", np.nan, inplace=True)
auto.drop("Warranty", axis=1, inplace=True)

#### "Make"

In [24]:
auto["Make"] = auto["Make"].str.strip("\n")

#### "Model" 

In [25]:
auto["Model"] = auto["Model"].str[1]

#### "First Registration" column has list and meaningles values.

In [26]:
auto["First Registration"] = auto["First Registration"].str[1]

#### "Body Color"

In [27]:
auto["Body Color"] = auto["Body Color"].str[1]

#### "Paint Type"

In [28]:
auto["Paint Type"] = auto["Paint Type"].str[0].str.strip("\n")

#### "Upholstery"

In [29]:
auto["Upholstery"] = auto["Upholstery"].str[0].str.strip("\n")

#### "Nr. of Doors"

In [30]:
auto["Nr. of Doors"] = auto["Nr. of Doors"].str[0].str.strip("\n")

#### "Nr. of Seats"

In [31]:
auto["Nr. of Seats"] = auto["Nr. of Seats"].str[0].str.strip("\n")

#### "Gearing Type" 

In [32]:
auto["Gearing Type"] = auto["Gearing Type"].str[1]

#### "Displacement" column has list and meaningless values.

In [33]:
auto["displacement(cc)"] = auto["Displacement"].str[0].str.strip("\n cc").str.replace(",","")
auto["displacement(cc)"] = pd.to_numeric(auto["displacement(cc)"])
auto.drop("Displacement", axis=1, inplace=True)

#### "Cylinders" 

In [34]:
auto["Cylinders"] = auto["Cylinders"].str[0].str.strip("\n")
auto["Cylinders"] = pd.to_numeric(auto["Cylinders"])

#### "Drive chain"

In [35]:
auto["Drive chain"] = auto["Drive chain"].str[0].str.strip("\n")

#### "Fuel" column is converted to gasoline types.

In [36]:
auto["Fuel"] = auto["Fuel"].str[1]

In [37]:
auto.loc[auto["Fuel"].notnull(), "Fuel"] = auto.loc[auto["Fuel"].notnull(), "Fuel"].apply(lambda x:"Diesel" if "diesel" in x.lower() else ("LPG/CNG" if (("lpg" in x.lower()) or ("cng" in x.lower())) else "Gasoline"))

####  "CO2 Emission" column has list and meaningless values.

In [38]:
auto["CO2 Emission"] = auto["CO2 Emission"].str[0]
auto["CO2 Emission"] = auto["CO2 Emission"].apply(lambda x:np.nan if type(x) == list else x)
auto["CO2 Emission(g/km (comb))"] = auto["CO2 Emission"].str.strip(" g CO2/km (comb)\n")
auto["CO2 Emission(g/km (comb))"] = auto["CO2 Emission(g/km (comb))"].str.replace(",","")
auto.drop("CO2 Emission", axis=1, inplace=True)

#### "Emission Class"  is converted to Euro 4-5-6 type.

In [39]:
auto["Emission Class"] = auto["Emission Class"].str[0].str.strip("\n")

In [40]:
auto["Emission Class"].value_counts()
auto.replace({"Emission Class" : {"Euro 6d-TEMP":"Euro 6", "Euro 6c":"Euro 6", "Euro 6d":"Euro 6"}}, inplace=True)

#### "\nComfort & Convenience\n" 

In [41]:
auto["Comfort & Convenience"] = [",".join(i) if type(i) == list else i for i in auto["\nComfort & Convenience\n"]]
auto.drop("\nComfort & Convenience\n", axis=1, inplace=True)

#### "\nEntertainment & Media\n" 

In [42]:
auto["Entertainment & Media"] = [",".join(i) if type(i) == list else i for i in auto["\nEntertainment & Media\n"]]
auto.drop("\nEntertainment & Media\n", axis=1, inplace=True)

#### "\nExtras\n" 

In [43]:
auto["Extras"] = [",".join(i) if type(i) == list else i for i in auto["\nExtras\n"]]
auto.drop("\nExtras\n", axis=1, inplace=True)

#### "\nSafety & Security\n"

In [44]:
auto["Safety & Security"] = [",".join(i) if type(i) == list else i for i in auto["\nSafety & Security\n"]]
auto.drop("\nSafety & Security\n", axis=1, inplace=True)

#### In "Emission Label" column we can choose either stickers or numbers.I decided to take emission label numbers.

In [45]:
auto["Emission Label"] = auto["Emission Label"].str[0].str.strip("\n").str.extract("(\d+)")

#### "Gears"

In [46]:
auto["Gears"] = auto["Gears"].str[0].str.strip("\n")

#### "Country version"

In [47]:
auto["Country version"] = auto["Country version"].str[0].str.strip("\n")

#### In "Consumption" column we have nested list values and it consists of three different columns. In order to get list values as string type, I created consumption function. Then I get three columns.

In [48]:
def consumption(x):
    if type(x)==list:
        if type(x[0]) == list:
            lst = list()
            for i in x:
                lst += i
            return " ".join(lst)
        else:
            return " ".join(x)

In [49]:
auto.loc[auto["Consumption"].notnull(),"Consumption"] = auto[auto["Consumption"].notnull()]["Consumption"].apply(consumption)
auto["Consumption(comb)"] = auto["Consumption"]
auto.loc[auto["Consumption(comb)"].notnull(),"Consumption(comb)"] = auto.loc[auto["Consumption(comb)"].notnull(),"Consumption(comb)"].apply(lambda x:np.nan  if "comb" not in x else re.findall("\d+\.*\,*\d*", x)[0])
auto["Consumption(comb)"] = pd.to_numeric(auto["Consumption(comb)"])

In [50]:
auto["Consumption(city)"] = auto["Consumption"]
auto.loc[auto["Consumption(city)"].notnull(),"Consumption(city)"] = auto.loc[auto["Consumption(city)"].notnull(),"Consumption(city)"].apply(lambda x: re.findall("\d+\.*\,*\d*(?= l/100 km \(city\))",x))
auto.loc[auto["Consumption(city)"].notnull(),"Consumption(city)"] = auto.loc[auto["Consumption(city)"].notnull(),"Consumption(city)"].apply(lambda x:np.nan if x == list() else x[0])
auto["Consumption(city)"] = pd.to_numeric(auto["Consumption(city)"])

In [51]:
auto["Consumption(country)"] = auto["Consumption"]
auto.loc[auto["Consumption(country)"].notnull(),"Consumption(country)"] = auto.loc[auto["Consumption(country)"].notnull(),"Consumption(country)"].apply(lambda x: re.findall("\d+\.*\,*\d*(?= l/100 km \(country\))",x))
auto.loc[auto["Consumption(country)"].notnull(),"Consumption(country)"] = auto.loc[auto["Consumption(country)"].notnull(),"Consumption(country)"].apply(lambda x:np.nan if x == list() else x[0])
auto["Consumption(country)"] = pd.to_numeric(auto["Consumption(country)"])
auto.drop("Consumption", axis=1, inplace = True)

#### "url", "short_description","Body Color Original","Model Code", "Full Service", "Non-smoking Vehicle", "null", "Offer Number", , "Weight", "description" columns will not effect decisions. Also,
#### "prev_owner","Body" columns have similar and more practical columns. They will be removed.

In [52]:
lst = ["url", "short_description", "prev_owner","Body Color Original","Model Code", "Full Service", "Non-smoking Vehicle", "null", "Offer Number", "Body", "Weight", "description"]
for i in lst:
    auto.drop(i, axis=1, inplace=True)

In [53]:
auto.head(3)

,make_model,body_type,price,vat,km,registration,Previous Owners,Next Inspection,Inspection new,Make,Model,First Registration,Body Color,Paint Type,Upholstery,Nr. of Doors,Nr. of Seats,Gearing Type,Cylinders,Drive chain,Fuel,Emission Class,Emission Label,Gears,Country version,hp(kW),condition_type,Warranty(month),displacement(cc),CO2 Emission(g/km (comb)),Comfort & Convenience,Entertainment & Media,Extras,Safety & Security,Consumption(comb),Consumption(city),Consumption(country)
0,Audi A1,Sedans,15770,VAT deductible,56013,2016-01-01,2,2021-06-01,1,Audi,A1,2016,Black,Metallic,"Cloth, Black",5,5,Automatic,3.0,front,Diesel,Euro 6,NaN,NaN,NaN,66.0,Used,NaN,1422.0,99,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,Hands-free equipment,On-board comput...","Alloy wheels,Catalytic Converter,Voice Control","ABS,Central door lock,Daytime running lights,D...",3.8,4.3,3.5
1,Audi A1,Sedans,14500,Price negotiable,80000,2017-03-01,NaN,NaT,0,Audi,A1,2017,Red,NaN,"Cloth, Grey",3,4,Automatic,4.0,front,Gasoline,Euro 6,4,7,NaN,141.0,Used,NaN,1798.0,129,"Air conditioning,Automatic climate control,Hil...","Bluetooth,Hands-free equipment,On-board comput...","Alloy wheels,Sport seats,Sport suspension,Voic...","ABS,Central door lock,Central door lock with r...",5.6,7.1,4.7
2,Audi A1,Sedans,14640,VAT deductible,83450,2016-02-01,1,NaT,0,Audi,A1,2016,Black,Metallic,"Cloth, Black",4,4,Automatic,NaN,front,Diesel,Euro 6,4,NaN,NaN,85.0,Used,NaN,1598.0,99,"Air conditioning,Cruise control,Electrical sid...","MP3,On-board computer","Alloy wheels,Voice Control","ABS,Central door lock,Daytime running lights,D...",3.8,4.4,3.4


In [54]:
auto.to_csv("cleanedauto.csv", index=False)